In [1]:
from __future__ import print_function, division # jupyter user interaction

from ipywidgets import interact, interactive, VBox, fixed, interact_manual, IntSlider # for userinteraction
import ipywidgets as widgets


%matplotlib qt5
import hyperspy.api as hs
# Don't worry about the warning below, they're just for information

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy import ndimage
from sklearn.neighbors import KDTree
from tqdm import tqdm_notebook
import pickle
from mpl_toolkits.mplot3d.art3d import Poly3DCollection # needed for 3d-plots
from mpl_toolkits.mplot3d import Axes3D  # needed for 3d-plots

from skimage import measure


#from __future__ import print_function, division
from scipy.optimize import brute, fminbound, minimize_scalar
# brute force minimizing
from itertools import combinations
#from numba import njit
from sklearn.neighbors import KDTree
from matplotlib.ticker import MultipleLocator, FormatStrFormatter

from tqdm import tqdm_notebook #########!!!!!!!!!!!!!!!!!


import numpy as np

from mpl_toolkits.mplot3d import Axes3D  # needed for 3d-plots
import matplotlib.pyplot as plt

from scipy.optimize import brute, fminbound, minimize_scalar
# brute force minimizing
from itertools import combinations
from ipywidgets import interact, interactive, VBox, fixed, interact_manual, IntSlider
#from numba import njit
from sklearn.neighbors import KDTree
from matplotlib.ticker import MultipleLocator, FormatStrFormatter

#from mpl_toolkits.mplot3d import proj3d # change perspective transformation


# import file
im = hs.load("LARBED_Stack_70mrad_EF_aligned.dm3")

#### get image parameters

kxoffset = im.axes_manager[1].offset # offset of image in 1/nm
kyoffset = im.axes_manager[2].offset
dkx =  im.axes_manager[1].scale # pixel scale in 1/nm
dky = im.axes_manager[2].scale
Nx = im.axes_manager.shape[1] # number of pixels
Ny = im.axes_manager.shape[2]
Nz = 51 # arbitrary, but not too big, because you might get empty voxels in 3d reconstruction
Nslice = im.axes_manager.shape[0] # number of stacked images

# wavelength lamda_e
emass = 510.99906;   # electron rest mass in keV
hce = 12.3984244;     # h*c/e  in A/keV
v0 = im.metadata.Acquisition_instrument.TEM.beam_energy # beam energy from metadata
lambda_e = 0.1 * hce/np.sqrt(v0*(2*emass+v0)) # wavelength in nm
k0 = 1/lambda_e # in 1/nm

#initialise 3d reciprocal Space cube 
cube = np.zeros((Nz,Nx,Ny))

# get tilt in rad
frames = im.original_metadata.ImageList.TagGroup0.ImageTags.QED.Frames

tx = frames[frames.keys()[0]].tiltX_mrad

for i in frames.keys()[1:]:
    tx = np.append(tx,[frames[i].tiltX_mrad])
    
ty = frames[frames.keys()[0]].tiltY_mrad

for i in frames.keys()[1:]:
    ty = np.append(ty,[frames[i].tiltY_mrad])
          

tilt = 0.001*np.array([tx,ty]).T # tilt array in rad
tMax = np.amax(np.sqrt(sum(tilt.T**2))) # maximum tilt angle in rad


# kz range in 1/nm
kzMax =  Nx/2*dkx*tMax
kzMin = -Nx/2*dkx*tMax
dkz = (kzMax-kzMin)/(Nz-1)


# kx, ky meshgrid and kz array 
kxl = np.arange(kxoffset, kxoffset + Nx*dkx, dkx)
kyl = np.arange(kyoffset, kyoffset + Ny*dky, dky)
kz = np.arange(kzMin,kzMax+dkz,dkz)
kx, ky = np.meshgrid(kxl, kyl)
    


# Test different thresholds and slices

In [2]:
def plot_with_threshold(thresh, slice_i, stdthresh):
    
    ########### preparing data ##################
    imi = im.inav[slice_i] # signal of  tilt with # slice_i
    imidata = imi.data # data array of current slice, transposed because [xindex,yindex]
    imi_ix = np.arange(imidata.shape[0])
    imi_iy = np.arange(imidata.shape[1])
    imi_ixmesh, imi_iymesh = np.meshgrid(imi_ix,imi_iy) # pixelindex mesh

    imi_x = kxoffset + dkx * imi_ix # array of x coordinates in recipricol units
    imi_y = kyoffset + dky * imi_iy #          y
    imi_xmesh, imi_ymesh =  np.meshgrid(imi_x,imi_y) # coordinates mesh

    thetaX = tilt[slice_i,0]
    thetaY = tilt[slice_i,1]
    
    # create binary mask of areas > threshold
    bin_mask = (imi.data > thresh)

    # imi_labeled gives array where the connected areas are labled with imi_labels labels
    imi_labeled, imi_labels = ndimage.label(bin_mask)

    # calculating center of intensity and standard deviation of labeled areas
    coi_ind_exact = np.array(ndimage.center_of_mass(imi.data, labels=imi_labeled, index=range(imi_labels + 1))) #center of mass index subpixel
    std = np.array(ndimage.standard_deviation(imi.data, labels=imi_labeled, index=range(imi_labels + 1)))
    coi_ind_exact[:, 0], coi_ind_exact[:, 1] = coi_ind_exact[:, 1], coi_ind_exact[:, 0].copy() # swap rows that x is in 0 and y in 1
    xcoiscal = kxoffset + dkx * coi_ind_exact.T[0]
    ycoiscal = kyoffset + dky * coi_ind_exact.T[1]
    coi = np.vstack((xcoiscal,ycoiscal)).T

    # take just COIs with standard deviation > stdthresh
    #stdthresh = 1
    stdmask =  std > stdthresh
    coi_ind_exact = coi_ind_exact[stdmask]
    coi = coi[stdmask]
    ############### exlude area around the center ##################################
    rad = 1 # radius of the excluded area
    radmask = np.sqrt(coi.T[0]**2 + coi.T[1]**2) >  rad
    coi = coi[radmask]
    coi_ind_exact = coi_ind_exact[radmask]
    coi_ind = np.array(np.round(coi_ind_exact).astype(int))

    fig = plt.figure(figsize=(8,8))
    # Use Latex
    plt.rc('text', usetex=True)
    plt.rc('font', family='serif')
    
    plt.imshow(imi.data, vmin = 0, vmax = 100, extent=[kxoffset , kxl.max(), kyl.max(), kyoffset])   
    figtitle = r"COI with threshold = " + str(thresh) + " and std-threshold = " + str(stdthresh)
    plt.title(figtitle)
    plt.xlabel(r'$k_{x}\,in\,\frac{1}{nm}$', fontsize = 20)
    plt.ylabel(r'$k_{y}\,in\,\frac{1}{nm}$', fontsize = 20)
    plt.plot(coi.T[0], coi.T[1], 'ro', markersize=4)  
    plt.colorbar()
    plt.show()
    
    return

print(r"You can control the threshold, standard deviation of the background and slice by sliding the slider")
stdthreshslider = IntSlider(value = 1, min=0, max=5,  description=r'\(standard\ deviation\)', style={'description_width': 'initial'})
threshslider = IntSlider(value = 50, min=5, max=200, step=1, description=r'\(intensity\ threshold\)', style={'description_width': 'initial'})
sliceslider = IntSlider(value = 20, min=0, max= Nslice, description=r'\(slice\)', style={'description_width': 'initial'})

interact_manual(plot_with_threshold, slice_i = sliceslider,  thresh = threshslider, stdthresh = stdthreshslider); 

You can control the threshold, standard deviation of the background and slice by sliding the slider


interactive(children=(IntSlider(value=50, description='\\(intensity\\ threshold\\)', max=200, min=5, style=Sli…

# test background substraction for every coi

In [3]:
# initialise test parameters
thresh = threshslider.value
slice_i = sliceslider.value
stdthresh = stdthreshslider.value

plotbgfit = True # Do you want to see the plot of the fitted background surface for one coi coi_i?
plotpeakfit = True # Do you want to see the plot of a peak coi_i and the fitted surface?

########### preparing data ##################
imi = im.inav[slice_i] # signal of  tilt with # slice_i
imidata = imi.data # data array of current slice, transposed because [xindex,yindex]
imi_ix = np.arange(imidata.shape[0])
imi_iy = np.arange(imidata.shape[1])
imi_ixmesh, imi_iymesh = np.meshgrid(imi_ix,imi_iy) # pixelindex mesh

imi_x = kxoffset + dkx * imi_ix # array of x coordinates in recipricol units
imi_y = kyoffset + dky * imi_iy #          y
imi_xmesh, imi_ymesh =  np.meshgrid(imi_x,imi_y) # coordinates mesh

thetaX = tilt[slice_i,0]
thetaY = tilt[slice_i,1]


# create binary mask of areas > threshold
bin_mask = (imi.data > thresh)

# imi_labeled gives array where the connected areas are labled with imi_labels labels
imi_labeled, imi_labels = ndimage.label(bin_mask)

# calculating center of intensity and standard deviation of labeled areas
coi_ind_exact = np.array(ndimage.center_of_mass(imi.data, labels=imi_labeled, index=range(imi_labels + 1))) #center of mass index subpixel
std = np.array(ndimage.standard_deviation(imi.data, labels=imi_labeled, index=range(imi_labels + 1)))
coi_ind_exact[:, 0], coi_ind_exact[:, 1] = coi_ind_exact[:, 1], coi_ind_exact[:, 0].copy() # swap rows that x is in 0 and y in 1
xcoiscal = kxoffset + dkx * coi_ind_exact.T[0]
ycoiscal = kyoffset + dky * coi_ind_exact.T[1]
coi = np.vstack((xcoiscal,ycoiscal)).T

# take just COIs with standard deviation > stdthresh
#stdthresh = 1
stdmask =  std > stdthresh
coi_ind_exact = coi_ind_exact[stdmask]
coi = coi[stdmask]



############### exlude area around the center ##################################
#rad = 1 # radius of the excluded area

#radmask = np.sqrt(coi.T[0]**2 + coi.T[1]**2) >  rad
#coi = coi[radmask]
#coi_ind_exact = coi_ind_exact[radmask]

coi_ind = np.array(np.round(coi_ind_exact).astype(int))

def test_coi(coi_i, peakrad, bgrad):


    coix_i = coi_ind[coi_i][0] # gives current center of intensity index x
    coiy_i = coi_ind[coi_i][1]

    ############## select peakarea ################################
    # get data of peak and surrounding area
    #peakrad = 8 # index radius around peak
    peakmask = peakrad**2 > (coix_i - imi_ixmesh)**2 + (coiy_i - imi_iymesh)**2 

    peakarea = imidata[peakmask]
    peakareaix = imi_ixmesh[peakmask] # circle index x
    peakareaiy = imi_iymesh[peakmask] #              y
    peakareax = kxoffset + dkx * peakareaix # coordinates
    peakareay = kyoffset + dky * peakareaiy

    ################ select backgroundarea ######################
    # get the surrounding area around the peak to approximate the background
    #bgrad = 4
    bgmask = bgrad**2 < (coix_i - peakareaix)**2 + (coiy_i - peakareaiy)**2 
    bgarea = peakarea[bgmask]
    bgareaix = peakareaix[bgmask]
    bgareaiy = peakareaiy[bgmask]
    bgareax = kxoffset + dkx * bgareaix # coordinates 1/nm
    bgareay = kyoffset + dky * bgareaiy


    ################ select peak ######################

    peakmask = ~bgmask
    peak = peakarea[peakmask]
    peakix = peakareaix[peakmask]
    peakiy = peakareaiy[peakmask]
    peakx = kxoffset + dkx * peakix # coordinates 1/nm
    peaky = kyoffset + dky * peakiy

    ################ Fitting a plane surface #################

    # regular grid covering the domain of the data

    X,Y = np.meshgrid(np.arange(bgareax.min(), bgareax.max()+dkx, dkx), np.arange(bgareay.min(),bgareay.max() + dky, dky))
    XX = X.flatten()
    YY = Y.flatten()

    # best-fit linear plane
    A = np.c_[bgareax, bgareay, np.ones(len(bgarea))]
    C,_,_,_ = sp.linalg.lstsq(A, bgarea)    # coefficients

    # evaluate it on grid
    Z = C[0]*X + C[1]*Y + C[2]

    # evaluate for peak
    bgpeakfit = C[0]*peakx + C[1]*peaky + C[2]

    # substract approximated background from peak
    bgredpeak = peak - bgpeakfit
    bgredpeak[bgredpeak < 0] = 0 # set all negative intensities to zero



    if plotbgfit == True:
        # plot points and fitted surface
        fig1 = plt.figure(figsize=(6, 5))
        ax = fig1.add_subplot(111, projection='3d') 
        ax.plot_surface(X, Y, Z, rstride=1, cstride=1, alpha=0.5, color = "blue")#, extend3d=True)
        bg3d = ax.scatter3D(bgareax,bgareay, bgarea, s=30, c=bgarea, marker = "o", vmin=0, vmax=bgarea.max(), depthshade=False)
        
        plt.rc('text', usetex=True)
        plt.rc('font', family='serif')
        #cbar=plt.colorbar(bg3d)
        #cbar.set_label("counts")
        plt.xlabel(r'$k_x\ in\ \frac{1}{nm}$', fontsize=18, labelpad=10)
        plt.ylabel(r'$k_y\ in\ \frac{1}{nm}$', fontsize=18, labelpad=10)
        ax.set_zlabel(r'$Counts$')
        ax.axis('equal')
        ax.axis('tight')
        plt.show()

    if plotpeakfit == True:
        fig2 = plt.figure(figsize=(6, 5))
        ax = fig2.add_subplot(111, projection='3d') 
        # background
        ax.plot_surface(X, Y, Z, rstride=1, cstride=1, alpha=0.5, color = "blue")
        # peakarea
        peak3d = ax.scatter3D(peakareax,peakareay, peakarea, s=30, c=peakarea, marker = "o", vmin=0, vmax=peakarea.max()+5000, depthshade=False)
        
        plt.rc('text', usetex=True)
        plt.rc('font', family='serif')
        #cbar=plt.colorbar(peak3d)
        #cbar.set_label("counts")
        plt.xlabel(r'$k_x\ in\ \frac{1}{nm}$', fontsize=18, labelpad=10)
        plt.ylabel(r'$k_y\ in\ \frac{1}{nm}$', fontsize=18, labelpad=10)
        ax.set_zlabel(r'$Counts$')
        ax.axis('equal')
        ax.axis('tight')
        plt.show()

print(r"You can test the background subtraction for an area around a certain COI of slice_i with outer and inner radius")


coislider = IntSlider(value = 60 , min=0, max= len(coi), description=r'\(COI\ index\)', style={'description_width': 'initial'})
peakradslider = IntSlider(value = 8 , min=2, max= 10, description=r'\(outer\ radius\ in\ pixels\)', style={'description_width': 'initial'})
bgradslider = IntSlider(value = 4 , min=0, max= 6, description=r'\(inner\ radius\ in\ pixels\)', style={'description_width': 'initial'})

interact_manual(test_coi, coi_i = coislider, peakrad = peakradslider, bgrad = bgradslider); 


You can test the background subtraction for an area around a certain COI of slice_i with outer and inner radius


interactive(children=(IntSlider(value=60, description='\\(COI\\ index\\)', max=80, style=SliderStyle(descripti…

# Loop over all Slices and COIs

$$k_{z}\left(k_{x,y},\theta_{x,y},\lambda\right)=\frac{1}{\lambda}\sqrt{1-\theta_{x}^{^{2}}-\theta_{y}^{^{2}}}-\sqrt{\frac{1}{\lambda^{2}}\left(1-\theta_{x}^{^{2}}-\theta_{y}^{^{2}}\right)-k_{x}^{^{2}}-k_{y}^{^{2}}+\frac{2\left(k_{x}\theta_{x}+k_{y}\theta_{y}\right)}{\lambda}}$$


In [8]:
#initialise
peakrad = peakradslider.value
bgrad = bgradslider.value



for slice_i in tqdm_notebook(range(0,Nslice)):

    ########### preparing data ##################
    imi = im.inav[slice_i] # signal of  tilt with # slice_i
    imidata = imi.data # data array of current slice, transposed because [xindex,yindex]
    imi_ix = np.arange(imidata.shape[0])
    imi_iy = np.arange(imidata.shape[1])
    imi_ixmesh, imi_iymesh = np.meshgrid(imi_ix,imi_iy) # pixelindex mesh

    imi_x = kxoffset + dkx * imi_ix # array of x coordinates in recipricol units
    imi_y = kyoffset + dky * imi_iy #          y
    imi_xmesh, imi_ymesh =  np.meshgrid(imi_x,imi_y) # coordinates mesh

    thetaX = tilt[slice_i,0]
    thetaY = tilt[slice_i,1]


    # create binary mask of areas > threshold (arbitrary, but if too low you get fake coi, too high some are not gonna be found)
    #thresh = imidata.mean()*5
    bin_mask = (imi.data > thresh)

    # imi_labeled gives array where the connected areas are labled with imi_labels labels
    imi_labeled, imi_labels = ndimage.label(bin_mask)

    # calculating center of intensity and standard deviation of labeled areas
    coi_ind_exact = np.array(ndimage.center_of_mass(imi.data, labels=imi_labeled, index=range(imi_labels + 1))) #center of mass index subpixel
    std = np.array(ndimage.standard_deviation(imi.data, labels=imi_labeled, index=range(imi_labels + 1)))
    coi_ind_exact[:, 0], coi_ind_exact[:, 1] = coi_ind_exact[:, 1], coi_ind_exact[:, 0].copy() # swap rows that x is in 0 and y in 1
    xcoiscal = kxoffset + dkx * coi_ind_exact.T[0]
    ycoiscal = kyoffset + dky * coi_ind_exact.T[1]
    coi = np.vstack((xcoiscal,ycoiscal)).T


    # take just COIs with standard deviation > stdthresh
    #stdthresh = 1
    stdmask =  std > stdthresh
    coi_ind_exact = coi_ind_exact[stdmask]
    coi = coi[stdmask]



    ############### exlude area around the center ##################################
    #rad = 1 # radius of the excluded area

    #radmask = np.sqrt(coi.T[0]**2 + coi.T[1]**2) >  rad
    #coi = coi[radmask]
    #coi_ind_exact = coi_ind_exact[radmask]

    coi_ind = np.array(np.round(coi_ind_exact).astype(int))

    ############################### center of intensitiy loop ##################
    for coi_i in range(0,len(coi)):

        coix_i = coi_ind[coi_i][0] # gives current center of intensity index x
        coiy_i = coi_ind[coi_i][1]

        ############## selecet peakarea ################################
        # get data of peak and surrounding area
        #peakrad = 8 # index radius around peak
        peakmask = peakrad**2 > (coix_i - imi_ixmesh)**2 + (coiy_i - imi_iymesh)**2 

        peakarea = imidata[peakmask]
        peakareaix = imi_ixmesh[peakmask] # circle index x
        peakareaiy = imi_iymesh[peakmask] #              y
        peakareax = kxoffset + dkx * peakareaix # coordinates
        peakareay = kyoffset + dky * peakareaiy

        ################ select backgroundarea ######################

        # get the surrounding area around the peak to approximate the background
        #bgrad = 4
        bgmask = bgrad**2 < (coix_i - peakareaix)**2 + (coiy_i - peakareaiy)**2 

        bgarea = peakarea[bgmask]
        bgareaix = peakareaix[bgmask]
        bgareaiy = peakareaiy[bgmask]
        bgareax = kxoffset + dkx * bgareaix # coordinates 1/nm
        bgareay = kyoffset + dky * bgareaiy


        ################ select peak ######################

        peakmask = ~bgmask

        peak = peakarea[peakmask]
        peakix = peakareaix[peakmask]
        peakiy = peakareaiy[peakmask]
        peakx = kxoffset + dkx * peakix # coordinates 1/nm
        peaky = kyoffset + dky * peakiy

        ################ Fitting a plane surface #################

        # regular grid covering the domain of the data

        X,Y = np.meshgrid(np.arange(bgareax.min(), bgareax.max(), dkx), np.arange(bgareay.min(),bgareay.max(), dky))
        XX = X.flatten()
        YY = Y.flatten()

        # best-fit linear plane
        A = np.c_[bgareax, bgareay, np.ones(len(bgarea))]
        C,_,_,_ = sp.linalg.lstsq(A, bgarea)    # coefficients

        # evaluate it on grid
        Z = C[0]*X + C[1]*Y + C[2]

        # evaluate for peak
        bgpeakfit = C[0]*peakx + C[1]*peaky + C[2]

        # substract approximated background from peak
        bgredpeak = peak - bgpeakfit
        bgredpeak[bgredpeak < 0] = 0 # set all negative intensities to zero

        kzOffs = np.cos(np.sqrt(thetaX**2+thetaY**2)) 
        #kzpeakarea = ((kzOffs-np.cos(np.sqrt((thetaX+lambda_e*peakx)**2+(thetaY+lambda_e*peaky)**2)))/lambda_e-kzMin)/dkz
        kzpeakarea = (1/lambda_e*np.sqrt(1-thetaX**2-thetaY**2)-np.sqrt(1/lambda_e**2*(1-thetaX**2-thetaY**2)-(peakx**2+peaky**2-2*(peakx*thetaX/lambda_e+peaky*thetaY/lambda_e)))-kzMin)/dkz
        kzpeakareai = np.round(kzpeakarea).astype(int) # round
        maskkz = ((kzpeakareai < Nz) & (kzpeakareai >= 0.)) # is calculated kz in the initial cube?
        kxi = peakix[maskkz]
        kyi = peakiy[maskkz]
        kzi = kzpeakareai[maskkz]
        cube[kzi,kxi,kyi] = cube[kzi,kxi,kyi] + bgredpeak[maskkz] #add the background substracted intensitiy at the right place of the cube

# save cube to a file
#np.save("cube.npy",cube)

# Plot of Data

In [17]:
plotcube = True # Do you want to see the Plot

#read cube from file if previously calculated (cube_old_forumluar.npy is the data calculated with the previous formular)
cube_load = pickle.load( open( "cube.pkl", "rb" ) ) # optional if not calculated
cube = cube_load # this overwrites the cube data calculated in "Loop over all Slices and COIs"

# Use marching cubes to obtain the surface mesh of 
verts, faces, normals, values = measure.marching_cubes_lewiner(cube, level=cube.mean()*10, spacing=(1,1,1), step_size=2)

# convert to 1/nm and coordinates of experiment
vertsnm = verts
vertsnm.T[0] = kzMin + dkz*verts.T[0]
vertsnm.T[1] = kxoffset + dkx*verts.T[1]
vertsnm.T[2] = kyoffset + dky*verts.T[2]




if plotcube == True:

    # Display resulting triangular mesh using Matplotlib. This can also be done
    # with mayavi (see skimage.measure.marching_cubes_lewiner docstring).
    fig = plt.figure(figsize=(6, 5))
    ax = fig.add_subplot(111, projection='3d', proj_type = 'ortho')


    # Fancy indexing: `verts[faces]` to generate a collection of triangles
    mesh = Poly3DCollection(vertsnm[faces])
    mesh.set_edgecolor('k')
    mesh.set_facecolor('w')
    ax.add_collection3d(mesh)
    
    # Use Latex
    plt.rc('text', usetex=True)
    plt.rc('font')

    ax.set_xlabel(r'$k_z\ in\ \frac{1}{nm}$', fontsize=18, labelpad=10)
    ax.set_ylabel(r'$k_x\ in\ \frac{1}{nm}$',fontsize=18, labelpad=10)
    ax.set_zlabel(r'$k_y\ in\ \frac{1}{nm}$',fontsize=18, labelpad=10)

    ax.set_xlim(kzMin, kzMax)  
    ax.set_ylim(kxoffset, kxoffset + Nx*dkx) 
    ax.set_zlim(kyoffset, kyoffset + Ny*dky)
    
    #equal axes
    #ax.set_xlim(kxoffset, (kxoffset + Nx*dkx))  
    #ax.set_ylim(kxoffset, (kxoffset + Nx*dkx)) 
    #ax.set_zlim(kxoffset, (kxoffset + Nx*dkx))
    
    ax.view_init(elev=45, azim=90)

    plt.tight_layout()
    plt.show() 

# Autocorrelation

In [31]:
autoplot = True # Do you want to see the Plot of the autocorrelation?

# Autocorrelation COI
autocoiplot = False # Do you want to see the Plot of the autocorrelation coi?

# autocorrelation

cfft = np.fft.fftn(cube)
cfft2 = cfft*np.conjugate(cfft)
cifft = np.fft.ifftn(cfft2)
cubeauto = np.abs(cifft)
cubeauto = np.fft.ifftshift(cubeauto)

# the threshhold should be high enough to get the cois in the middle of the cube but not too high so you lose to many cois
cubethresh = cubeauto.mean()*50

  # Use marching cubes to obtain the surface mesh of 
verts, faces, normals, values = measure.marching_cubes_lewiner(cubeauto, level=50*cubeauto.mean(), spacing=(1,1,1), step_size=2)

# convert to 1/nm and coordinates of experiment
vertsnm = verts
vertsnm.T[0] = kzMin + dkz*verts.T[0]
vertsnm.T[1] = kxoffset + dkx*verts.T[1]
vertsnm.T[2] = kyoffset + dky*verts.T[2]


if autoplot == True:

  
    
    # Display resulting triangular mesh using Matplotlib. This can also be done
    # with mayavi (see skimage.measure.marching_cubes_lewiner docstring).
    fig = plt.figure(figsize=(6, 6))
    ax = fig.add_subplot(111, projection='3d', proj_type = 'ortho')


    # Fancy indexing: `verts[faces]` to generate a collection of triangles
    mesh = Poly3DCollection(vertsnm[faces])
    mesh.set_edgecolor('k')
    mesh.set_facecolor('w')
    ax.add_collection3d(mesh)

    ax.set_xlabel(r'$k_z\ in\ \frac{1}{nm}$', fontsize=18, labelpad=10)
    ax.set_ylabel(r'$k_x\ in\ \frac{1}{nm}$',fontsize=18, labelpad=10)
    ax.set_zlabel(r'$k_y\ in\ \frac{1}{nm}$',fontsize=18, labelpad=10)

    ax.set_xlim(kzMin, kzMax)  
    ax.set_ylim(kxoffset, kxoffset + Nx*dkx) 
    ax.set_zlim(kyoffset, kyoffset + Ny*dky)
    

    plt.tight_layout()
    plt.show()

    
# create binary mask of areas > threshold
cube_bin_mask = (cubeauto > cubethresh)

# imi_labeled gives array where the connected areas are labled with imi_labels labels
cube_labeled, cube_labels = ndimage.label(cube_bin_mask)

# calculating center of intensity and standard deviation of labeled areas
autocoi = np.array(ndimage.center_of_mass(cubeauto, labels=cube_labeled, index=range(cube_labels + 1))) #center of mass index subpixel

autocoi.T[1] = kxoffset + dkx * autocoi.T[1]
autocoi.T[2] = kyoffset + dky * autocoi.T[2]
autocoi.T[0] = kzMin + dkz*autocoi.T[0]



#################### Plot of COIs of autocorrelation ######################

if autocoiplot == True:

    fig = plt.figure(figsize=(6,6))
    ax = fig.add_subplot(111, projection='3d', proj_type = 'ortho')


    ax.scatter(autocoi.T[1], autocoi.T[2], autocoi.T[0], c='k', marker='o',s=5)

    ax.set_xlabel(r'$k_z\ in\ \frac{1}{nm}$', fontsize=18, labelpad=10)
    ax.set_ylabel(r'$k_x\ in\ \frac{1}{nm}$',fontsize=18, labelpad=10)
    ax.set_zlabel(r'$k_y\ in\ \frac{1}{nm}$',fontsize=18, labelpad=10)
    plt.show()
    

In [35]:
# save cois of autocorrelation to a file (optional)
#np.save("autocoi.npy",autocoi)

In [19]:
if autocoiplot == True:

    fig = plt.figure(figsize=(8,8))
    ax = fig.add_subplot(111, projection='3d', proj_type = 'ortho')
    ax.view_init(elev=270, azim=90)


    ax.scatter(autocoi.T[1], autocoi.T[2], autocoi.T[0], c='k', marker='o',s=5)

    ax.set_xlabel(r'$k_x\ in\ \frac{1}{nm}$', fontsize=18, labelpad=10)
    ax.set_ylabel(r'$k_y\ in\ \frac{1}{nm}$',fontsize=18, labelpad=10)
    ax.set_zlabel(r'$k_z\ in\ \frac{1}{nm}$',fontsize=18, labelpad=10)
    plt.show()

# Find Primitive Cell

In [20]:
# define needed functions:

def distances(coord):
    '''
    calculate all appearing distances in an array of coordinates
    '''
    n = coord.shape[0]
    result = []
    for i in range(n):
        for j in range(i):
            result.append(np.linalg.norm((coord[i] - coord[j])))
    return np.array(result)


def dist_pnt(coord, point):
    '''
    calculate all distances of an array of coordinates to a single point
    '''
    return np.sqrt((coord[:, 0]-point[0])**2 +
                   (coord[:, 1]-point[1])**2 +
                   (coord[:, 2]-point[2])**2)


def first_val(hist, bin_edges, bound):
    '''
    get the first distance value that appears more often then given in "bounds"

    as input you can use the output of numpy.histogram
    '''
    return bin_edges[:-1][hist > bound][0]


def ang_to_norm(phi, psi):
    '''
    get a normal-vector for the hessian-normal form of a plane given by two
    angles (phi - angle to positive z-axes, psi - angle to positive x-axes)
    '''
    return np.array([np.sin(phi)*np.cos(psi),
                     np.sin(phi)*np.sin(psi),
                     np.cos(phi)])


def dist_to_plane(points, phi, psi, dzero):
    '''
    calculate the distance of a point to a plane, where the plane is given by
    phi, psi, and dzero (distance to the origin)
    '''
    points = np.array(points, ndmin=2)
    norm = ang_to_norm(phi, psi)
    result = np.empty(points.shape[0])

#    for i in range(points.shape[0]):
#        result[i] = abs(np.dot(norm, points[i]) - dzero) !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

    result = np.abs(norm[0]*points[:, 0] +
                    norm[1]*points[:, 1] +
                    norm[2]*points[:, 2] - dzero)
    return result


def dist_to_plane_sgn(points, phi, psi, dzero):
    '''
    calculate the distance of a point to a plane, where the plane is given by
    phi, psi, and dzero (distance to the origin) (with sign)
    '''
    points = np.array(points, ndmin=2)
    norm = ang_to_norm(phi, psi)
    result = np.empty(points.shape[0])

#    for i in range(points.shape[0]):
#        result[i] = abs(np.dot(norm, points[i]) - dzero)

    result = (norm[0]*points[:, 0] +
              norm[1]*points[:, 1] +
              norm[2]*points[:, 2] - dzero)
    return result


def dist_to_line(points, norm, vec):
    '''
    distance of points to a given line by x = t*n + v where n is a normal-vec

    returns : distances and the parameter t of each nearest point

    optimal t is given by: t_min = norm*(point - vec)
    '''
    para = (norm[0]*(points[:, 0] - vec[0]) +
            norm[1]*(points[:, 1] - vec[1]) +
            norm[2]*(points[:, 2] - vec[2]))

    result = np.sqrt((points[:, 0] - norm[0]*para - vec[0])**2 +
                     (points[:, 1] - norm[1]*para - vec[1])**2 +
                     (points[:, 2] - norm[2]*para - vec[2])**2)

    return result, para


def rescale_basis_vec(points, norm, vec, eps=None):
    '''
    return the points on the line within an eps-surrounding
    '''
    if eps is None:
        dist_coord = distances(points)
        # use the histogram to find the minimum distance between the points
        min_dist = first_val(*np.histogram(dist_coord, bins=points.shape[0]))
        # set the epsilon for checking if point in plane
        eps = min_dist/10

    dists, paras = dist_to_line(points, norm, vec)
    # select the t-parameters of the nearest points on line
    paras = paras[dists < eps]
    paras = np.sort(paras)
    # the consecutive distances of the params
    cons_dist = paras[1:]-paras[:-1]

#    print("consecutive distances", cons_dist)

    return np.mean(cons_dist)


def no_of_points(in_arg, points, eps, blist=None, angl=np.pi/3.0):
    '''
    get the number of points whose distance to the given plane is less than eps

    you can "blacklist" certain angles for psi and phi in blist
    with "angl" you can specify the angle around the blacklisted angles
    (stnd value: 45°)

    if an angle-combination is blacklisted, 0 is returned
    '''
    if blist is None:
        blist = []

    phi, psi, dzero = in_arg

    # iterate over blacklisted angles
    for bphi, bpsi in blist:
        if (bphi-angl < phi < bphi+angl) and (bpsi-angl < psi < bpsi+angl):
            return 0

    dist = dist_to_plane(points, phi, psi, dzero)
    # result is negative so the minimizer is working as a maximizer
    return -len(dist[dist < eps])


def center_unit(points):
    '''
    center and scale a list of points to unitcube and give the shift and
    scaling
    
    the scaling is done to numerical stabilise the optimisation

    shifting is realized by the average position

    return : newpoints, scale, shift
    '''
    coord = points.copy()
    shift = np.mean(coord, axis=0)
    for i in range(3):
        coord[:, i] -= shift[i]
    scale = np.max(np.abs(coord), axis=0)
    for i in range(3):
        coord[:, i] /= scale[i]

    return coord, scale, shift


def affin_hessian(norm, dist, matrix, vec=np.array([0., 0., 0.])):
    '''
    transformation of a plane given in hessian normal form (n*x=d) under a
    affine transformation: y = T(x) = M*x + v
    '''
    # inverse matrix of the transformation
    inv_mat = np.linalg.inv(matrix)
    # the new normal-vector n_new = n*M^-1
    new_norm = np.squeeze(np.dot(np.array(norm, ndmin=2), inv_mat))
    abs_norm = np.linalg.norm(new_norm)
    new_norm = new_norm/abs_norm

    new_dist = dist/abs_norm + np.dot(new_norm, vec)

    return new_norm, new_dist


def estimate_planes(points, iterations=4,
                    error_angle=np.pi/3.0,
                    grid_angle=np.pi/90):
    '''
    performe estimations to lay planes in a cloud of points

    with iterations you can specifiy the amount of planes

    with error_angle you can set the angle of minimal difference between planes

    with grid_angle you can specify the grid-size for the brute-force estimate
    '''
    
    coord = points.copy()
    # empty list for results
    resbrute = []
    # scale the coords
    coord, scale, shift = center_unit(coord)
    #print("scale", scale)
    #print("shift", shift)
    # get the distances within the coords
    dist_coord = distances(coord)
    
    
    # use the histogram to find the minimum distance between the points
    min_dist = first_val(*np.histogram(dist_coord, bins=coord.shape[0]), bound = bounds)
    # set a maximal range where the plane can move from the origin
    max_dist = np.max(dist_coord)/8.0
    # set the epsilon for checking if point in plane
    eps = min_dist/8.0
    #print("min_dist", min_dist)
    #print("max_dist", max_dist)
    #print("eps", eps)
    # initialize the black-list
    blist = []
    for i in range(iterations):
        print("----- iteration: {}".format(i))
        res = brute(no_of_points,
                    ranges=(slice(0., np.pi/2., grid_angle),  # 90° in 2° steps
                            slice(0., 2.*np.pi, grid_angle),  # 360° in 2° step
                            slice(-max_dist, max_dist, eps/2.5)),
                    args=(coord, eps, blist, error_angle),
                    disp=False)
        blist.append(res[:2])
        #print("phi, psi, dist", res)
        resbrute.append(res)

    # components of the affine backtransformation for the coords
    #    trans_mat = np.diag(1/scale)
    trans_mat = np.diag(scale)
    trans_vec = shift

    # get the normal-vectors and dists of the planes from their angles
    norms = []
    dists = []
    for res in resbrute:
        norms.append(ang_to_norm(res[0], res[1]))
        dists.append(res[2])

    out = []

    # backtransform the planes to the original coords
    for norm, dist in zip(norms, dists):
        out.append(affin_hessian(norm, dist, trans_mat, trans_vec))

    return out


def plot_planes(coord, planes, vectors=None, origin=None, cubesize = 6):
    '''
    plot the choord-cloud and the planes given as a list of norms and dists
    '''
    # create the figure for plotting
    fig = plt.figure()
    
    plt.rc('text', usetex=True)
    plt.rc('font', family='serif')
    # create the first axis
    ax = fig.add_subplot(111, projection='3d', proj_type = 'ortho')
    # plot the points
    ax.scatter(coord[:, 0], coord[:, 1], coord[:, 2], c='k' )
    
    
    ax.set_xlabel(r'$k_z\ in\ \frac{1}{nm}$')
    ax.set_ylabel(r'$k_x\ in\ \frac{1}{nm}$')
    ax.set_zlabel(r'$k_y\ in\ \frac{1}{nm}$')

    # get the axes-limits to cut the plane afterwards
    x_lim = ax.get_xlim()
    y_lim = ax.get_ylim()
    z_lim = ax.get_zlim()

    # set 30 steps for the plane in x and y direction
    x_stp = (x_lim[1] - x_lim[0])/30
    y_stp = (y_lim[1] - y_lim[0])/30
    # generate the mesh for plotting the plane
    xx, yy = np.meshgrid(np.arange(x_lim[0], x_lim[1], x_stp),
                         np.arange(y_lim[0], y_lim[1], y_stp))
    # get the normal vector of the plane
    for normal, dist in planes:
        # calculate the z-elevation of the planes
        zz = (-normal[0]*xx - normal[1]*yy + dist)/normal[2]
 
        ax.plot_surface(xx, yy, zz, color='b', alpha=0.05) # color=(1, 0, 0, 0.3)
    ax.set_xlim([-cubesize/10, cubesize/10])
    ax.set_ylim([-cubesize, cubesize])
    ax.set_zlim([-cubesize, cubesize])
    # plot vectors
    if vectors is not None:
        X = [origin[0], origin[0], origin[0]]
        Y = [origin[1], origin[1], origin[1]]
        Z = [origin[2], origin[2], origin[2]]
        U = [vectors[0][0], vectors[1][0], vectors[2][0]]
        V = [vectors[0][1], vectors[1][1], vectors[2][1]]
        W = [vectors[0][2], vectors[1][2], vectors[2][2]]
        ax.quiver(X, Y, Z, U, V, W)
    # show the plot
    plt.show()


def normal_basis(plane1, plane2, plane3):
    '''
    calculate a normal-basis from the intersections of 3 planes:
    b1, b2, b3, origin
    '''
    #print("------------")
    #print("basis vectors")
    b1 = np.cross(plane1[0], plane2[0])
    b1 = b1/np.linalg.norm(b1)
    b2 = np.cross(plane2[0], plane3[0])
    b2 = b2/np.linalg.norm(b2)
    b3 = np.cross(plane3[0], plane1[0])
    b3 = b3/np.linalg.norm(b3)

    matrix = np.vstack((plane1[0], plane2[0], plane3[0]))
    rhs = np.array((plane1[1], plane2[1], plane3[1]))
    origin = np.linalg.solve(matrix, rhs)

    #print("basis_1", b1)
    #print("basis_2", b2)
    #print("basis_3", b3)
    #print("origin", origin)

    return b1, b2, b3, origin


def new_coord(coord, b1, b2, b3, ori):
    '''
    get the new coordinates of a list of points for a given new coordiante-sys
    with basis vectors b1,b2,b3 and the origin ori (where the planes intersect)
    '''
    # the basis-change matrix
    mat = np.linalg.inv(np.vstack((b1, b2, b3)).T)
    new_coord = np.empty_like(coord)
    # calculate the new coords by multipling coord with the basis-change matrix
    for i in range(coord.shape[0]):
        new_coord[i] = np.squeeze(np.dot(mat,
                                         np.array(coord[i]-ori, ndmin=2).T))
    return new_coord

######################## 3 minimization routine

def mae_to_int(length, new_coord, component=0):
    '''
    calculate the mean absolut error to a grid with given length of the given
    "component"-th basis-vector
    '''
    arr = new_coord[:, component]/length
    return np.mean(np.abs(arr-np.rint(arr)))


def get_basis_lengths(coord, b1, b2, b3, ori):
    '''
    given a suitable basis in normal form, calculate fitting lengths of
    the basis-vectors, so that the coords are lying in an integer-grid
    '''
    # get the distances within the coords
    dist_coord = distances(coord)
    # use the histogram to find the minimum distance between the points
    min_dist = first_val(*np.histogram(dist_coord, bins=coord.shape[0]), bound =  bounds) /2
    # set a maximal range where the plane can move from the origin
    max_dist = np.max(dist_coord)/7.0
    #print("min_dist", min_dist)
    #print("max_dist", max_dist)
    new_coor = new_coord(coord, b1, b2, b3, ori)
    lengths = []
    # optimize over the length of each basis vector
    for i in range(3):
        #print("----------------------")
        #print("scaling basis vector", i)
        length = fminbound(mae_to_int, min_dist, max_dist,
                           args=(new_coor, i),
                           xtol=1e-5,
                           maxfun=50000,
                           disp=False)
        # check if a multiple of the actual length fits also
        # if bigger 
        error = mae_to_int(length, new_coor, i)
        #print("error", error)
        scale = 1
        for j in range(2, 5):
            error_i = mae_to_int(j*length, new_coor, i)
            if abs(error - error_i) > min_dist/2:
                #print("error_diff", abs(error - error_i))
                #print("scaling", j-1)
                scale = j-1
                break

        lengths.append(scale*length)

    return lengths

def get_combi_iter(iterat):
    ''' 
    given a suitable basis in normal form, calculate fitting lengths of
    the basis-vectors, so that the coords are lying in an integer-grid
    '''
    comb = combinations(range(iterat), 3)
    combi = [] 
    for i in list(comb):
        combi.append(i)
    return np.array(combi)

def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """  
    
    if np.linalg.norm(v1) == 0 or np.linalg.norm(v2) == 0:
        return 0
    
    else:
        v1_u = unit_vector(v1)
        v2_u = unit_vector(v2)
        return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))
    

In [21]:
# plane approach
if __name__ == "__main__":
    # load the points
    coord = np.load("autocoi_thresh_50.npy") # optional, takes calculated data of the autocorrelation
    
    bounds = len(coord)/2

    ###########    USERINTERACTION ###############
    iterat = 4  # number of iterations (number of planes that are calculated)
                # estimate 4 planes in hessian normal form given by angles of norm-vector
                # (phi, psi) and distance to origin    
    
    planes = estimate_planes(coord, iterations=iterat);
    # plane 1 and 2 are very similar (take plane 2 in the following)
    # get the normal-basis from plane 0, 2 and 3
    

----- iteration: 0
----- iteration: 1
----- iteration: 2
----- iteration: 3


In [22]:
combi = get_combi_iter(iterat) # all combination of planes
sumbasvec = []
basangles = []

################# USERINTERACTION ################
combi_n = 3 # plane combination [1, 2, 3]


p1, p2, p3 = combi[combi_n]
b1, b2, b3, origin = normal_basis(planes[p1], planes[p2], planes[p3]);
scale = get_basis_lengths(coord, b1, b2, b3, origin);

# rescale the basis
basis1 = scale[0]*b1
basis2 = scale[1]*b2
basis3 = scale[2]*b3

# planes for the shortes sum of basis vectors
#p1, p2, p3 = combi[sumbasvec == np.min(sumbasvec)][0] # get plane combination with minimal sum of basis vecotrs


# plot the planes and basis-vectors

plot_planes(coord, [planes[p1], planes[p2], planes[p3]],
            vectors=[basis1, basis2, basis3],
            origin=origin)
# plot the coordinates in the integer grid
int_coord = np.empty_like(coord)


# get the new coordinates for the new basis
new_coor = new_coord(coord, basis1, basis2, basis3, origin)

# round new coordinates to integer values
new_coor = np.array(np.rint(new_coor), dtype=int)

for i in range(3):
    int_coord[:, i] = (new_coor[:, 0]*basis1[i] +
                       new_coor[:, 1]*basis2[i] +
                       new_coor[:, 2]*basis3[i] +
                       origin[i])

# basis angles
alpha1 = angle_between(basis1, basis2)
alpha2 = angle_between(basis1, basis3)
alpha3 = angle_between(basis2, basis3)




# save all results
np.savetxt("basis1.txt", basis1)
np.savetxt("basis2.txt", basis2)
np.savetxt("basis3.txt", basis3)
np.savetxt("origin.txt", origin)
np.savetxt("new_coord.txt", new_coor, fmt="%d")

plot_planes(int_coord, [planes[p1], planes[p2], planes[p3]],
            vectors=[basis1, basis2, basis3],
            origin=origin)

# Data Plot

In [26]:
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

cubesize = 6 # size of plotted cube
vectors = [basis1, basis2, basis3] # vectors to be plotted
origin = origin # define origin of the plot

# create the figure for plotting
fig = plt.figure()
# create the first axis
ax = fig.add_subplot(111, projection='3d', proj_type = 'ortho')


# plot the points
bg3d = ax.scatter(coord[:, 0], coord[:, 1], coord[:, 2], c='k' )
# get the axes-limits to cut the plane afterwards
x_lim = ax.get_xlim()
y_lim = ax.get_ylim()
z_lim = ax.get_zlim()

ax.set_xlabel(r'$k_{z}\,in\,\frac{1}{nm}$',fontsize = 20)
ax.set_ylabel(r'$k_{x}\,in\,\frac{1}{nm}$',fontsize = 20)
ax.set_zlabel(r'$k_{y}\,in\,\frac{1}{nm}$',fontsize = 20)


# set 30 steps for the plane in x and y direction
x_stp = (x_lim[1] - x_lim[0])/30
y_stp = (y_lim[1] - y_lim[0])/30
# generate the mesh for plotting the plane
xx, yy = np.meshgrid(np.arange(x_lim[0], x_lim[1], x_stp),
                     np.arange(y_lim[0], y_lim[1], y_stp))
# get the normal vector of the plane
for normal, dist in planes:
    # calculate the z-elevation of the planes
    zz = (-normal[0]*xx - normal[1]*yy + dist)/normal[2]

    ax.plot_surface(xx, yy, zz, color='b', alpha=0.075) # color=(1, 0, 0, 0.3)

ax.set_xlim([-cubesize/10, cubesize/10])
ax.set_ylim([-cubesize, cubesize])
ax.set_zlim([-cubesize, cubesize])
# plot vectors
if vectors is not None:
    X = [origin[0], origin[0], origin[0]]
    Y = [origin[1], origin[1], origin[1]]
    Z = [origin[2], origin[2], origin[2]]
    U = [vectors[0][0], vectors[1][0], vectors[2][0]]
    V = [vectors[0][1], vectors[1][1], vectors[2][1]]
    W = [vectors[0][2], vectors[1][2], vectors[2][2]]
    ax.quiver(X, Y, Z, U, V, W, color='b')
# show the plot
plt.show()

#figname = "data" + str(ang1) + "-" + str(ang2) + ".pdf"
#fig.savefig(figname, bbox_inches='tight')

/home/till/anaconda3/lib/python3.6/site-packages/matplotlib/font_manager.py:1241: UserWarning: findfont: Font family ['serif'] not found. Falling back to DejaVu Sans.
  (prop.get_family(), self.defaultFamily[fontext]))


# Simulation plot

In [27]:
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

cubesize = 6
vectors = [basis1, basis2, basis3]
origin = origin

# create the figure for plotting
fig = plt.figure()
# create the first axis
ax = fig.add_subplot(111, projection='3d', proj_type = 'ortho')



# plot the points
bg3d = ax.scatter(int_coord[:, 0], int_coord[:, 1], int_coord[:, 2], c='k' )
# get the axes-limits to cut the plane afterwards
x_lim = ax.get_xlim()
y_lim = ax.get_ylim()
z_lim = ax.get_zlim()


ax.set_xlabel(r'$k_{z}\,in\,\frac{1}{nm}$',fontsize = 20)
ax.set_ylabel(r'$k_{x}\,in\,\frac{1}{nm}$',fontsize = 20)
ax.set_zlabel(r'$k_{y}\,in\,\frac{1}{nm}$',fontsize = 20)


# set 30 steps for the plane in x and y direction
x_stp = (x_lim[1] - x_lim[0])/30
y_stp = (y_lim[1] - y_lim[0])/30
# generate the mesh for plotting the plane
xx, yy = np.meshgrid(np.arange(x_lim[0], x_lim[1], x_stp),
                     np.arange(y_lim[0], y_lim[1], y_stp))
# get the normal vector of the plane
for normal, dist in planes:
    # calculate the z-elevation of the planes
    zz = (-normal[0]*xx - normal[1]*yy + dist)/normal[2]

    ax.plot_surface(xx, yy, zz, color='b', alpha=0.075) # color=(1, 0, 0, 0.3)

ax.set_xlim([-cubesize/10, cubesize/10])
ax.set_ylim([-cubesize, cubesize])
ax.set_zlim([-cubesize, cubesize])
# plot vectors
if vectors is not None:
    X = [origin[0], origin[0], origin[0]]
    Y = [origin[1], origin[1], origin[1]]
    Z = [origin[2], origin[2], origin[2]]
    U = [vectors[0][0], vectors[1][0], vectors[2][0]]
    V = [vectors[0][1], vectors[1][1], vectors[2][1]]
    W = [vectors[0][2], vectors[1][2], vectors[2][2]]
    ax.quiver(X, Y, Z, U, V, W, color='b')
    # show the plot
plt.show()

In [28]:
angb12 = angle_between(basis1,basis2)*180/np.pi
angb23 = angle_between(basis2,basis3)*180/np.pi
angb13 = angle_between(basis1,basis3)*180/np.pi
print("Angel between basis 12, 23, 13:")
print(angb12, angb23, angb13)


nb1 = np.linalg.norm(basis1)
nb2 = np.linalg.norm(basis2)
nb3 = np.linalg.norm(basis3)

print("nb1, nb2, nb3:")
print(nb1, nb2, nb3)

Angel between basis 12, 23, 13:
92.51315915408244 16.790909398742148 75.82213705292297
nb1, nb2, nb3:
1.2774142823217831 2.673145237644728 2.7123315542042348


# Plot basis vectors

In [23]:
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

#cubesize = 6
vectors = [basis1, basis2, basis3]

# create the figure for plotting
fig = plt.figure()
# create the first axis
ax = fig.add_subplot(111, projection='3d', proj_type = 'ortho')


# plot the points
bg3d = ax.scatter(int_coord[:, 0], int_coord[:, 1], int_coord[:, 2], c='k' )


ax.set_xlabel(r'$k_{z}\,in\,\frac{1}{nm}$',fontsize = 20)
ax.set_ylabel(r'$k_{x}\,in\,\frac{1}{nm}$',fontsize = 20)
ax.set_zlabel(r'$k_{y}\,in\,\frac{1}{nm}$',fontsize = 20)



limit1 = -1.5
limit2 = 1.5
ax.set_xlim([origin[0] + limit1, origin[0] + limit2])
ax.set_ylim([origin[1] + limit1, origin[1] + limit2])
ax.set_zlim([origin[2] + limit1, origin[2] + limit2])
# plot vectors
if vectors is not None:
    X = [origin[0], origin[0], origin[0]]
    Y = [origin[1], origin[1], origin[1]]
    Z = [origin[2], origin[2], origin[2]]
    U = [vectors[0][0], vectors[1][0], vectors[2][0]]
    V = [vectors[0][1], vectors[1][1], vectors[2][1]]
    W = [vectors[0][2], vectors[1][2], vectors[2][2]]
    ax.quiver(X, Y, Z, U, V, W, color='blue')
    # show the plot
plt.show()

# Smallest, linear independent basis vector ansatz

In [24]:
# import int_coord
int_coord= np.loadtxt("int_coord.txt")

kdt = KDTree(int_coord, leaf_size=2, metric='euclidean') # neighbors
distances, indices = kdt.query(int_coord, k=len(int_coord)) # distances and indices all cois



index = 4 # index of an appropriate origin point

smdist = distances[index]
smind = index
#for i in range(len(smdist)):
#    while distances[i,j] <= smdist[j]:
#            smdist = distances[i]
#            smind = i
#            i =+ 1


#find nearest neighbor of beginning point
for i in range(len(smdist)):
    if smdist[i] > 0:
        dist_min = smdist[i]
        break
        
indi_min = indices[smind][smdist == dist_min]
prim_basis1 = int_coord[indi_min][0] - int_coord[indices[smind,0]] # first basis vector

#find second nearest neighbor of first lattice point
for i in range(len(smdist)):
    if smdist[i] > dist_min:
        dist_min2 = smdist[i]
        
        indi_min2 = indices[smind][smdist == dist_min2]
        prim_basis2 = int_coord[indi_min2][0] - int_coord[indices[smind,0]] # second basis vector
        
        #check if linear independent      
        A = np.row_stack([prim_basis1, prim_basis2])
        U, s, V = np.linalg.svd(A)
        
        if np.sum(s > 0.0000001) > 1 :      # rank of matrix must be bigger than 1 for the vectors to be
                                            # linear independent
            break

#find third nearest neighbor of first lattice point
for i in range(len(distances[0])):
    if smdist[i] > dist_min2:
        dist_min3 = smdist[i]
        
        indi_min3 = indices[smind][smdist == dist_min3]
        prim_basis3 = int_coord[indi_min3][0] - int_coord[indices[smind,0]] # third basis vector
        #check if linear independent      
        A = np.row_stack([prim_basis1, prim_basis2, prim_basis3])
        U, s, V = np.linalg.svd(A)
        
        if np.sum(s > 0.0001) > 2:          # rank of matrix must be bigger than 2 for the vectors to be
                                            # linear independent
            break

angnb12 = angle_between(prim_basis1,prim_basis2)*180/np.pi
angnb23 = angle_between(prim_basis2,prim_basis3)*180/np.pi
angnb13 = angle_between(prim_basis1,prim_basis3)*180/np.pi
print("Angel between basis 12, 23, 13:")
print(angnb12, angnb23, angnb13)


nb1 = np.linalg.norm(prim_basis1)
nb2 = np.linalg.norm(prim_basis2)
nb3 = np.linalg.norm(prim_basis3)

print("nb1, nb2, nb3:")
print(nb1, nb2, nb3)

Angel between basis 12, 23, 13:
93.00050901803708 94.68947943203788 100.6216418920945
nb1, nb2, nb3:
0.342674488935843 0.3531766025541926 2.4421544907358093


# Plot of primitive basis

In [28]:

plt.rc('text', usetex=True)
plt.rc('font', family='serif')

#cubesize = 6
vectors = [prim_basis1, prim_basis2, prim_basis3]
origin_prim = int_coord[smind]

# create the figure for plotting
fig = plt.figure()
# create the first axis
ax = fig.add_subplot(111, projection='3d', proj_type = 'ortho')


# plot the points
bg3d = ax.scatter(int_coord[:, 0], int_coord[:, 1], int_coord[:, 2], c='k' )


ax.set_xlabel(r'$k_{z}\,in\,\frac{1}{nm}$',fontsize = 20)
ax.set_ylabel(r'$k_{x}\,in\,\frac{1}{nm}$',fontsize = 20)
ax.set_zlabel(r'$k_{y}\,in\,\frac{1}{nm}$',fontsize = 20)


#ax.set_xlim([int_coord[smind,0] - 1, int_coord[smind,0] + 1.5])
#ax.set_ylim([int_coord[smind,1] -1.5, int_coord[smind,1] + 1])
#ax.set_zlim([int_coord[smind,2] - 2, int_coord[smind,2] + 0.5])

limit1 = -2
limit2 = 2
ax.set_xlim([int_coord[smind,0] + limit1, int_coord[smind,0] + limit2])
ax.set_ylim([int_coord[smind,1] + limit1, int_coord[smind,1] + limit2])
ax.set_zlim([int_coord[smind,2] + limit1, int_coord[smind,2] + limit2])
# plot vectors
if vectors is not None:
    X = [origin_prim[0], origin_prim[0], origin_prim[0]]
    Y = [origin_prim[1], origin_prim[1], origin_prim[1]]
    Z = [origin_prim[2], origin_prim[2], origin_prim[2]]
    U = [vectors[0][0], vectors[1][0], vectors[2][0]]
    V = [vectors[0][1], vectors[1][1], vectors[2][1]]
    W = [vectors[0][2], vectors[1][2], vectors[2][2]]
    ax.quiver(X, Y, Z, U, V, W, color='blue')
    # show the plot
plt.show()

# Indexing

In [33]:
# get the new coordinates for the new basis
new_prime_coor = new_coord(int_coord, prim_basis1, prim_basis2, prim_basis3, origin_prim)

# round new coordinates to integer values
new_prime_coor = np.array(np.rint(new_prime_coor), dtype=int)

# save all results
np.savetxt("prim_basis1.txt", prim_basis1)
np.savetxt("prim_basis2.txt", prim_basis2)
np.savetxt("prim_basis3.txt", prim_basis3)
np.savetxt("origin_prim.txt", origin_prim)
np.savetxt("new_prime_coor.txt", new_prime_coor, fmt="%d")
